In [2]:
import ee
import geemap

In [9]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)
Map

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [40]:
def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': 250,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':250,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':250,
        'bestEffort':True,
        'maxPixels':1000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':250,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':250,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':250,
    'bestEffort':True,
    'maxPixels':1000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    

In [11]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(bottom=35226.0, center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [22]:
reducers = ee.Reducer.mean()


# Reducer para cálculo do EVI médio de cada cluster
stats = lst.reduceRegion(**{
  'reducer': reducers,
  'geometry':region,
  'scale':250,
});

In [24]:
print(stats.getInfo())

{'NDVI': 8108.787373042909}


In [62]:
    #Filtrar a territorialidade 
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate('2020-01-01', '2021-04-30').filter(ee.Filter.calendarRange(1, 4,'month')).map(clusterization)
    #4 principais meses de safra
    #filter start and end date
    
    

In [43]:
import pandas as pd

In [63]:
a = mcollection.aggregate_array('EVI_Cluster')

In [64]:
x = a.getInfo()

EEException: Computation timed out.

In [ ]:
df = pd.DataFrame({'lin':x})

In [83]:
import requests

#API Estação INMET
#Recuperar dados da estação mais próxima ao geocode passado como parâmetro:
#https://apiprevmet3.inmet.gov.br/estacao/proxima/<geocode>



#table = []



import json

geocode = '2919553'

base_site = 'https://apiprevmet3.inmet.gov.br/estacao/proxima/' + geocode

r = requests.get(base_site)

json = r.json()

dt = pd.DataFrame(json['dados'],index=[0])

estacao = dt['CD_ESTACAO'].values[0]

base_site2 = 'https://apitempo.inmet.gov.br/estacao/diaria/2020-01-01/2020-12-31/' + estacao

r = requests.get(base_site2)

json = r.json()

dt2 = pd.DataFrame(json)


In [84]:
dt2

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE
0,82.6,2020-01-01,BARREIRAS,56,24.8,3.4,-12.12472221,21.5,30.6,BA,1.7,A402,-45.02694443
1,89.8,2020-01-02,BARREIRAS,81,23.8,1.4,-12.12472221,22.5,26.4,BA,1.5,A402,-45.02694443
2,90.9,2020-01-03,BARREIRAS,67,23.3,10.6,-12.12472221,21.2,28.9,BA,1.6,A402,-45.02694443
3,95.8,2020-01-04,BARREIRAS,83,22.9,9.2,-12.12472221,22.3,25.3,BA,0.9,A402,-45.02694443
4,89.6,2020-01-05,BARREIRAS,67,23.5,56,-12.12472221,20.6,28.5,BA,1.5,A402,-45.02694443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,74.2,2020-12-27,BARREIRAS,47,26.4,None,-12.12472221,21.2,32.2,BA,1.3,A402,-45.02694443
362,73.3,2020-12-28,BARREIRAS,44,25.8,None,-12.12472221,19.5,32.3,BA,0.7,A402,-45.02694443
363,72,2020-12-29,BARREIRAS,39,26.3,None,-12.12472221,20.3,33.6,BA,0.8,A402,-45.02694443
364,66,2020-12-30,BARREIRAS,39,26.8,None,-12.12472221,20,33.9,BA,1,A402,-45.02694443
